In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import csv
import json
import numpy as np
import pymongo

In [4]:
# Read CSV into dataframe
brew_csv = "../data/beer_reviews.csv"
brewer_df = pd.read_csv(brew_csv)

craft_csv = "../data/craft_beer_fest_data2.csv"
craft_df = pd.read_csv(craft_csv)

# rename columns for merging later
craft_df.rename(columns={'Beer':'beer_name','Brewery':'brewery_name'}, inplace=True)

#drop duplicates from this data set
craft_df = craft_df.drop_duplicates(subset="beer_name", keep='first', inplace=False)

FileNotFoundError: File b'../data/craft_beer_fest_data2.csv' does not exist

In [3]:
group_by_beer_name_mean = brewer_df.groupby('beer_name').mean()
group_by_beer_name_mean = group_by_beer_name_mean.reset_index()
# group_by_beer_name_mean.head(20)

al_beer_df = pd.merge(group_by_beer_name_mean, craft_df, on=["beer_name"], how='outer')
al_beer_df = al_beer_df.dropna(how='any').reset_index(drop=True)
al_beer_df = al_beer_df.drop(columns=['brewery_id', 'review_time','Table','Ratings','ABV','URL','beer_beerid','Score','Bro Score'])

# print(al_beer_df.columns.values)

avg_scores = list(al_beer_df["review_overall"])
avg_overall_scores = list(al_beer_df["rAvg"])
weighted_averages = []

for av, ov in zip(avg_scores, avg_overall_scores):
    weighted_avg = (av+ov)/2
    weighted_averages.append(weighted_avg)

al_beer_df['Avg Overall Score'] = weighted_averages
al_beer_df = al_beer_df.drop(columns=['review_overall', 'rAvg'])
al_beer_df



,beer_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv,brewery_name,Location,Style,Avg Overall Score
0,Hefeweizen,4.0,3.0,3.833333,3.833333,5.0,Cambridge Brewing Company,"Cambridge, MA",Hefeweizen,4.075


In [92]:
def get_average_scores(column, dfToFilter):
    group_by_current = dfToFilter.groupby(column).mean()

    counts_df = dfToFilter[column].value_counts().to_frame(name=None).reset_index()
    counts_df = counts_df.rename(index=str, columns={"index": column, column: "Quantity"})

    with_counts_df = pd.merge(group_by_current, counts_df, on=[column], how='outer')
    with_counts_df = with_counts_df.rename(index=str, columns={"review_aroma":"Aroma","review_appearance":"Appearance",
                                                                       "review_palate":"Palate","review_taste":"Taste",
                                                                       "beer_abv":"ABV(%)"})
    return with_counts_df

In [93]:
grouped_df = get_average_scores("City", al_beer_df)
grouped_df


,State,Aroma,Appearance,Palate,Taste,ABV(%),Avg Overall Score,Quantity
0,CA,3.921843,3.964798,3.930602,4.017240,7.248889,4.001448,21
1,CO,3.780891,3.852190,3.772604,3.802633,6.896499,3.831093,16
2,CT,4.139376,4.236842,4.260234,4.299220,8.000000,4.235010,1
3,DC,3.808140,3.901495,3.851627,3.831355,6.555556,3.831725,3
4,DE,3.923901,3.889020,3.806561,3.932592,8.666667,3.798015,3
5,HI,3.257247,3.296533,3.394309,3.397613,5.000000,3.462338,2
6,IL,4.046084,3.992735,4.016158,4.084817,9.320676,4.040061,3
7,LA,3.219735,3.391133,3.238185,3.254318,4.470303,3.377024,2
8,MA,3.749273,3.876456,3.785743,3.819075,6.703172,3.860424,43
9,MD,4.057062,4.080226,3.964972,4.051412,8.300000,3.960311,1
